In [86]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import os
from datasets import load_dataset
from transformers import DataCollatorForSeq2Seq
from datasets import load_metric
import numpy as np
from transformers import Seq2SeqTrainingArguments
from transformers import Seq2SeqTrainer


In [87]:
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

In [102]:

model_checkpoint = "facebook/nllb-200-distilled-600M"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, src_lang='kor_Hang', tgt_lang='eng_Latn')
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)
model.config.forced_bos_token_id = tokenizer.convert_tokens_to_ids('eng_Latn')
# model.config.bos_token_id = tokenizer.convert_tokens_to_ids('eng_Latn')

In [89]:
dataset = load_dataset('json', data_files='./ai_hub/1.Training/원천데이터/일상생활및구어체_한영_train_set.json', field='data')

split_datasets = dataset["train"].train_test_split(train_size=0.9, seed=20)
split_datasets["validation"] = split_datasets.pop("test")

# declare preprocessing code 
max_input_length = 128
max_target_length = 128

def preprocess_function(examples):
    inputs = examples['ko']
    targets = examples['en']

    model_inputs = tokenizer(inputs, text_target=targets, max_length=max_input_length, truncation=True)

    return model_inputs            


Found cached dataset json (/home/juhwan/.cache/huggingface/datasets/json/default-3c8e71bae241100f/0.0.0/fe5dd6ea2639a6df622901539cb550cf8797e5a6b2dd7af1cf934bed8e233e6e)
100%|██████████| 1/1 [00:00<00:00, 24.85it/s]
Loading cached split indices for dataset at /home/juhwan/.cache/huggingface/datasets/json/default-3c8e71bae241100f/0.0.0/fe5dd6ea2639a6df622901539cb550cf8797e5a6b2dd7af1cf934bed8e233e6e/cache-0e94f1019394202e.arrow and /home/juhwan/.cache/huggingface/datasets/json/default-3c8e71bae241100f/0.0.0/fe5dd6ea2639a6df622901539cb550cf8797e5a6b2dd7af1cf934bed8e233e6e/cache-f8893aa3fa6006a7.arrow


In [90]:
tokenized_datasets = split_datasets.map(
    preprocess_function,
    batched=True,
    remove_columns=split_datasets["train"].column_names,
)

Loading cached processed dataset at /home/juhwan/.cache/huggingface/datasets/json/default-3c8e71bae241100f/0.0.0/fe5dd6ea2639a6df622901539cb550cf8797e5a6b2dd7af1cf934bed8e233e6e/cache-42da862947ade6a5.arrow
Loading cached processed dataset at /home/juhwan/.cache/huggingface/datasets/json/default-3c8e71bae241100f/0.0.0/fe5dd6ea2639a6df622901539cb550cf8797e5a6b2dd7af1cf934bed8e233e6e/cache-b72a71f3b5c6901b.arrow


In [91]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [92]:
metric = load_metric("sacrebleu")

In [93]:

def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]

    return preds, labels

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]

    
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)
    
    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result



In [94]:

args = Seq2SeqTrainingArguments(
    f"nllb-finetuned-aihub-ko-to-en",
    evaluation_strategy="no",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=64,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=3,
    predict_with_generate=True,
    fp16=True,
    push_to_hub=False,

)


In [95]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)


In [101]:
# # Testing
# article = split_datasets['validation'][0]['ko']
# inputs = tokenizer(article, return_tensors='pt')

# model.to('cpu')
# translated_tokens = model.generate(
#     # **inputs, forced_bos_token_id=tokenizer.lang_code_to_id["eng_Latn"], max_length=30
#     **inputs, max_length=30
# )


# tokenizer.batch_decode(translated_tokens,  skip_special_tokens=True)[0]

'食事はレストランだけでできるのか?'

In [96]:
# 학습하기전 초기모델 점수
trainer.evaluate(max_length=max_target_length)

You're using a NllbTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/data/juhwan/nlp-task-translation-en-ko/.venv/lib/python3.8/site-packages/transformers/generation/utils.py:1219: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(


{'eval_loss': 1.4868464469909668,
 'eval_bleu': 26.5843,
 'eval_gen_len': 16.7345,
 'eval_runtime': 2733.1555,
 'eval_samples_per_second': 43.905,
 'eval_steps_per_second': 0.686}

In [103]:
trainer.train()

Step,Training Loss


In [ ]:
trainer.evaluate(max_length=max_target_length)